<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_06_V2_SingleChart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ॐ श्री गणेशाय नमः

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

#Single Chart Analysis along with Gochar

##Rationale

The core 'engine' of Parashar21 is designed to cast horoscopes using the [Python](https://pypi.org/project/pyswisseph/) version of the [Swiss Ephemeris](https://www.astro.com/swisseph/) software, convert the data to JSON format and then store the same in a MongoDB database.<br><br> While processing a single chart does not really need database engine we found it economical to skip another ground up development and retain the same code base. Moreover, the Mongo query language has been used to filter out horoscopes that meet certain criteria and then create human readable MS-Word documents for each selected horoscope. This search and filter operation would have been difficult without the MongoDB engine. <br> <br>
For full information about this project, please see the [Parashar21](https://prithwis.github.io/parashar21/) github page.


#SetUp Environment

##Install Software

In [ ]:
#!python --version
#!lsb_release -a

#check which version of MongoDB  is available
#!apt-cache policy mongodb

#check which versions of software are available
#!pip3 index versions pymongo
#!pip3 index versions pyswisseph
#!pip3 index versions python-docx


In [2]:
!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
!pip -qq install python-docx                                # https://python-docx.readthedocs.io/en/latest/
#!/usr/bin/python3 -m pip install "pymongo[srv]"            # Required only if used with Atlas remote servers
!pip -qq install pymongo                                    # Required for Mongo client
!apt install mongodb > /dev/null                            # install local MongoDB server           

##Install, import prerequisites

In [3]:
from datetime import datetime
import pytz
print('Tested on ',datetime.now(pytz.timezone('Asia/Calcutta')))


#Utility functions 
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz
from google.colab import files

#import matplotlib.pyplot as plt
#import math


Tested on  2023-05-15 10:10:18.904461+05:30


In [4]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py      # report writing utility functions
!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py    # yog data - description and conditions 
# ---------------------------

#!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png                 
#!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png   



In [5]:
import p21
import p21utils
import p21utilsR
import p21swe
import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
importlib.reload(p21swe)
importlib.reload(p21YogInfo)

10 Yogs on record
0 : Amala Yog | 10th from Moon OR Lagna should have a benefic
1 : RajLakshana Yog | Jupiter, Venus, Mercury, Moon should be in Lagna or they should be in a Kendra
2 : Chatussagara Yog | All the Kendras 1/4/7/10 are occupied by planets, Ra, Ke not counted here
3 : Adhi678 Yog | Benefics located in 6/7/8th  from Moon
4 : Chandra Mangal Yog | Mars Conjoins Moon or  Mars and Moon in Mutual Aspect
5 : Kemadruma Yog : No planets on either sides of the Moon
6 : Anapha Yog |Planets other than Sun in 12th Place from Moon
7 : Sunapha Yog |Planets other than Sun in 2nd Place from Moon
8 : Dhurdhura Yog | Planets on both sides of the Moon
9 : Gajakesari Yog | Jupiter is in Kendra or 1,4,7,10th from Moon
10 Yogs on record
0 : Amala Yog | 10th from Moon OR Lagna should have a benefic
1 : RajLakshana Yog | Jupiter, Venus, Mercury, Moon should be in Lagna or they should be in a Kendra
2 : Chatussagara Yog | All the Kendras 1/4/7/10 are occupied by planets, Ra, Ke not counted here
3 :

<module 'p21YogInfo' from '/content/p21YogInfo.py'>

In [6]:
# Guess what is happening here :-)
#
!wc *.py | grep total |awk '{print $1 " lines";}'
!sed '/^#/d' *.py | wc |awk '{print $1 " non-comment lines";}'

2107 lines
1889 non-comment lines


In [7]:
#Download Swiss Ephemeris Files 
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe
#------------------------------
# Configure SWE with appropriate parameters
#
p21swe.C01_configSWE()

##Local Mongo

We use a locally installed, temporary MongoDB to store and process the data


In [8]:
!service mongodb start                                      # start Local MongoDB server instance
#!mongod --version

# Connect to local MongoDB

mongoURI = "mongodb://localhost:27017/"                               # for local server
client = MongoClient(mongoURI)
db = client.parashar21

 * Starting database mongodb
   ...done.


#Report Generation Function

In [9]:
def genChart(ChartStyle):
      
    chartData = chart.iloc[0]
    RepID = chart.iloc[0]['pid']['name']+'_'+p21.AnalysisType[0:3]+'_'+ChartStyle[0:1]
    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21_'+RepID+'_'+now.strftime("%H%M%S")+'.doc'

    #print(chartData)
    # ---------------------------------------------------------
    p21utilsR.R01_CreateReportDoc(None,None,'SingleChart')        # Creates MS Word Doc called p21.document for Single or MULTIPLE charts
    #p21utilsR.R01A_CreateReportDoc(RepID)                           # Creates MS Word Doc called p21.document for SINGLE chart
    # ----------------------------------------------------------
    
    p21utilsR.R511_parseChartData(chartData)                # Break up database data into useful pieces
                                                            # Creates two primary dicts, GLon, GRet
    p21utilsR.R30_LocateYogs()
    
    #print(p21.beneficG)
    for ctype in ['Rashi','Navamsa']:
        p21.ChartType = ctype
        p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                            # Rashi Num as well as Rashi name
        if ChartStyle == 'Bengal':
            p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
        if ChartStyle == 'South':
            p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
        if ChartStyle == 'North':
            p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
        
    
    p21utilsR.R512_FormatPage('SingleChart')                  # this is where the details of the chart are printed out
    
    # ------------------------------------
    p21.document.save(fileName)
    return fileName
    
print("Executed at ", datetime.now(pytz.timezone('Asia/Kolkata')))

Executed at  2023-05-15 10:10:55.335774+05:30


# Store and Retrieve Function

In [10]:
def StoreRetrieveChart():
    with open('peopleData.json') as json_file:
        oneChart = json.load(json_file)

    db.drop_collection('khona21')
    db.create_collection('khona21')               # Optional collection creation 
    p21.kollection = db.khona21                   # Set the collection to work with

    # Insert the single chart

    insert_result = p21.kollection.insert_one(oneChart)
    #insert_result.acknowledged  
    #chart = pd.DataFrame(list(p21.kollection.find({},p21.selCols)))
    return(pd.DataFrame(list(p21.kollection.find({},p21.selCols))))

print("Executed at ", datetime.now(pytz.timezone('Asia/Kolkata')))

Executed at  2023-05-15 10:11:05.960330+05:30


# Natal Chart
Data for others can be added by adding an extra line in the cell below. Please follow the same pattern as in the other lines


## Representative Natal Chart Data

In [ ]:
# Sample Data for some people
#
#%%writefile peopleData.csv
#Gender,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name,tag1,tag2,tag3,tag4,tag5,tag6
#M,17,9,1950,23:11,5.5,23.72,63.36,IST,standard,NDModi,Politician,nil,Administrator,nil,nil,nil
#F,10,10,1954,11:00,5.5,13.08,80.27,IST,standard,Rekha,Actor,nil,Dancer,nil,nil,nil
#M,15,10,1931,1:15,5.5,9.29,79.31,IST,standard,APJAbdulKalam,Engineer,nil,PublicFigure,nil,nil,nil
#F,19,11,1917,23:11,5.5,25.43,81.85,IST,standard,IndiraG,Politician,nil,PublicFigure,nil,nil,nil
#M,15,10,1961,23:42,5.5,22.57,88.37,IST,standard,Person1,Engineer,nil,Teacher,nil,PostGraduate,nil
#F,19,12,1965,10:26,5.5,22.57,88.37,IST,standard,Person1,Author,nil,Homemaker,nil,PostGraduate,nil
#M,9,10,1990,23:09,5.5,22.57,88.37,IST,standard,Person1,Engineer,nil,Teacher,nil,PostGraduate,nil
#F,12,9,1990,0:52,5.5,22.87,88.37,IST,standard,Person1,Biologist,nil,Scientist,nil,PostGraduate,nil

#Read Birth Data from a file
#!wget -O peopleData.csv -q https://raw.githubusercontent.com/prithwis/parashar21/main/data/Test5Data.txt # 5 Person Test Data
#!cat peopleData.csv

In [ ]:
# Get Lat Long for Place of Birth, in case that is not known
import requests
import urllib.parse

address = 'Vadnagar, Gujarat, India'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])
#print(response)

23.7853622
72.6332132


## Natal Data Preparation

In [11]:
%%writefile peopleData.csv
Gender,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name,tag1,tag2,tag3,tag4,tag5,tag6
F,19,12,1965,10:26,5.5,22.57,88.37,IST,standard,Person1,Author,nil,Homemaker,nil,PostGraduate,nil

Writing peopleData.csv


In [12]:
#%%time

df = pd.read_csv('peopleData.csv')
#df.head()

p21.AnalysisType = 'Rashi'                                     # one of ['Rashi','Navamsa']
#p21.AnalysisType = 'Navamsa'                                    # one of ['Rashi','Navamsa']
p21.ChartType = p21.AnalysisType

#p21.Gochar = False                                              # This should be set to False, in all cases

# ------------------------------------------------------------
# Converts date/time info into a detailed horoscope chart 
# Stores the same in a JSON file, to be inserted into a local MongoDB database
# 
p21swe.C61_Cast2JSON(df)
p21.SubMoonLong = p21.GLon['Mo']                       # Preserving Moon Longitude for Gochar
print(p21.GLon['La'],p21.GLon['Mo'])


1 records processed, so far
1  records generated and stored in file peopleData.json
310.415 201.001


##Process Natal Chart

In [13]:
%%capture

chart = StoreRetrieveChart()
ReportFile = genChart('Bengal')
#ReportFile = genChart('South')

## Dasha Coding

In [14]:
#print(p21.DashaDuraFract)
#print(p21.DashaDurationD)
#[2557,7305,2192,3653,2557,6575,5844,6940,6209]

[0.058333333333333334, 0.16666666666666666, 0.05, 0.08333333333333333, 0.058333333333333334, 0.15, 0.13333333333333333, 0.15833333333333333, 0.14166666666666666]
[2556.75, 7305.0, 2191.5, 3652.5, 2556.75, 6574.5, 5844.0, 6939.75, 6209.25]


In [15]:
def NextDasha(cd):
  cdIndex = p21.DashaSeq.index(cd)
  ndIndex = cdIndex+1
  if ndIndex > 8:
    ndIndex = 0
  return(p21.DashaSeq[ndIndex])

In [16]:
def GetDasha():
  #print(p21.DoB)
  #print(p21.NksGap)
  #print(p21.SubMoonLong)

  BirthNks = int(p21.SubMoonLong/p21.NksGap)                    # Number of Birth Nakshatra
  print('Birth Nakshatra ',BirthNks,p21.Nks[BirthNks])
  BirthDasha = p21.DashaStart[BirthNks]                         # BirthDasha
  print('BirthDasha ',BirthDasha)
  BirthDashaIndex = p21.DashaSeq.index(BirthDasha)
  print('Birth Dasha Index', BirthDashaIndex, )

  DashaConsumedDeg = p21.SubMoonLong - BirthNks*p21.NksGap
  #print(DashaConsumedDeg)
  #DashaConsumedDuration = round((DashaConsumedDeg/p21.NksGap) * p21.DashaDurationD[BirthDashaIndex])
  DashaConsumedDuration = (DashaConsumedDeg/p21.NksGap) * p21.DashaDurationD[BirthDashaIndex]          # Rounding Causes cumulative errors

  print('Birth Dasha Duration',p21.DashaDurationD[BirthDashaIndex], 'Birth Dasha Consumed Duration', DashaConsumedDuration )
  #DaysToStartOfBirthDasha = int(DashaConsumedDuration * 365)
  
  StartOfBirthDasha = p21.DoB - timedelta(days=DashaConsumedDuration)
  print('Start of Birth Dasha',StartOfBirthDasha)
  print('Date of Birth',p21.DoB)
  L1Dasha = BirthDasha
  StartOfL1Dasha = StartOfBirthDasha
  p21.VimDasha = {}
  for i in range(9):
    L1DashaDuration = p21.DashaDurationD[p21.DashaSeq.index(L1Dasha)]
    EndOfL1Dasha = StartOfL1Dasha + timedelta(days=L1DashaDuration)
    DashaDict = {}
    #datetime.strptime(time_data, format_data)
    #now.strftime("%m/%d/%Y, %H:%M:%S")
    DashaDict['Start'] = StartOfL1Dasha.strftime("%d %b %Y")
    DashaDict['End'] = EndOfL1Dasha.strftime("%d %b %Y")
    DashaDict['Duration'] = L1DashaDuration
    L2Dasha = L1Dasha
    StartOfL2Dasha = StartOfL1Dasha
    for j in range(9):
      AntarDashaDict = {}
      #L2DashaDuration = round(L1DashaDuration * p21.DashaDuraFract[p21.DashaSeq.index(L2Dasha)])
      L2DashaDuration = L1DashaDuration * p21.DashaDuraFract[p21.DashaSeq.index(L2Dasha)]

      #L2DashaDuration = L1DashaDuration * p21.DashaDuraFract[p21.DashaSeq.index(L2Dasha)]
      EndOfL2Dasha = StartOfL2Dasha + timedelta(days=L2DashaDuration)
      AntarDashaDict['Start'] = StartOfL2Dasha.strftime("%d %b %Y")
      AntarDashaDict['End'] = EndOfL2Dasha.strftime("%d %b %Y")
      AntarDashaDict['Duration'] = L2DashaDuration
      #AntarDashaDict['Antardasha'] = L2Dasha
      #AntarDashaDict[L2Dasha] = AntarDashaDictX
      DashaDict[L2Dasha] = AntarDashaDict
      L2Dasha = NextDasha(L2Dasha)
      StartOfL2Dasha = EndOfL2Dasha
    p21.VimDasha[L1Dasha] = DashaDict
    #print(i+1,'Dasha',Dasha,'Starts',StartOfDasha,'Ends',EndOfDasha,'Followed by',NextDasha(Dasha))
    L1Dasha = NextDasha(L1Dasha)
    StartOfL1Dasha = EndOfL1Dasha
  #print(VimDasha)
  #print(json.dumps(p21.VimDasha, indent = 4))
  #printVim()

In [17]:
GetDasha()

Birth Nakshatra  15 Vishakha
BirthDasha  Ju
Birth Dasha Index 6
Birth Dasha Duration 5844.0 Birth Dasha Consumed Duration 438.738300000002
Start of Birth Dasha 1964-10-06 16:42:50.880000
Date of Birth 1965-12-19 10:26:00


In [18]:
def printVim():
  for k1, v1 in p21.VimDasha.items():
    #print('outer')
    for k2, v2 in v1.items():
      if (type(v2) is dict):
        print(k1,' Dasha ends on ',v1['End'],' | ',k2,' Antardasha ends on ',v2['End'])
    Error = datetime.strptime(v1['End'],"%d %b %Y") - datetime.strptime(v2['End'],"%d %b %Y")
    print('----------- Error of ', Error.days,' days in ',v1['Duration'],' day Dasha')
   
 def printMahaDasha():
  for k1, v1 in p21.VimDasha.items():
    print(k1,' Dasha ends on ',v1['End'])

def printAntarDasha():
    #today = datetime.now(pytz.timezone('Asia/Calcutta'))
    today = datetime.now()
    print(today)
    count = 0
    for k1, v1 in p21.VimDasha.items():
        if (today < datetime.strptime(v1['End'],"%d %b %Y")) and (count < 2):
            if (count == 0):
                print('Current Dasha', count, k1)
            else:
                print('Next Dasha',count)
            #print('count', count)
            for k2, v2 in v1.items():
                if (type(v2) is dict):
                    print(k1,' Dasha ends ',v1['End'],' | ',k2,' Antardasha ends ',v2['End'])
                #Error = datetime.strptime(v1['End'],"%d %b %Y") - datetime.strptime(v2['End'],"%d %b %Y")
                #print('----------- Error of ', Error.days,' days in ',v1['Duration'],' day Dasha')
            count = count+1

In [20]:
printMahaDasha()

Ju  Dasha ends on  06 Oct 1980
Sa  Dasha ends on  07 Oct 1999
Me  Dasha ends on  06 Oct 2016
Ke  Dasha ends on  07 Oct 2023
Ve  Dasha ends on  07 Oct 2043
Su  Dasha ends on  06 Oct 2049
Mo  Dasha ends on  07 Oct 2059
Ma  Dasha ends on  07 Oct 2066
Ra  Dasha ends on  06 Oct 2084


In [47]:
printAntarDasha()

2023-05-15 04:56:34.829442
Current Dasha 0 Ke
Ke  Dasha ends  07 Oct 2023  |  Ke  Antardasha ends  04 Mar 2017
Ke  Dasha ends  07 Oct 2023  |  Ve  Antardasha ends  04 May 2018
Ke  Dasha ends  07 Oct 2023  |  Su  Antardasha ends  09 Sep 2018
Ke  Dasha ends  07 Oct 2023  |  Mo  Antardasha ends  10 Apr 2019
Ke  Dasha ends  07 Oct 2023  |  Ma  Antardasha ends  07 Sep 2019
Ke  Dasha ends  07 Oct 2023  |  Ra  Antardasha ends  24 Sep 2020
Ke  Dasha ends  07 Oct 2023  |  Ju  Antardasha ends  31 Aug 2021
Ke  Dasha ends  07 Oct 2023  |  Sa  Antardasha ends  10 Oct 2022
Ke  Dasha ends  07 Oct 2023  |  Me  Antardasha ends  07 Oct 2023
Next Dasha 1
Ve  Dasha ends  07 Oct 2043  |  Ve  Antardasha ends  05 Feb 2027
Ve  Dasha ends  07 Oct 2043  |  Su  Antardasha ends  06 Feb 2028
Ve  Dasha ends  07 Oct 2043  |  Mo  Antardasha ends  06 Oct 2029
Ve  Dasha ends  07 Oct 2043  |  Ma  Antardasha ends  07 Dec 2030
Ve  Dasha ends  07 Oct 2043  |  Ra  Antardasha ends  06 Dec 2033
Ve  Dasha ends  07 Oct 2043  | 

In [ ]:
printVim()

Ju  Dasha ends on  06 Oct 1980  |  Ju  Antardasha ends on  24 Nov 1966
Ju  Dasha ends on  06 Oct 1980  |  Sa  Antardasha ends on  07 Jun 1969
Ju  Dasha ends on  06 Oct 1980  |  Me  Antardasha ends on  13 Sep 1971
Ju  Dasha ends on  06 Oct 1980  |  Ke  Antardasha ends on  18 Aug 1972
Ju  Dasha ends on  06 Oct 1980  |  Ve  Antardasha ends on  19 Apr 1975
Ju  Dasha ends on  06 Oct 1980  |  Su  Antardasha ends on  06 Feb 1976
Ju  Dasha ends on  06 Oct 1980  |  Mo  Antardasha ends on  07 Jun 1977
Ju  Dasha ends on  06 Oct 1980  |  Ma  Antardasha ends on  14 May 1978
Ju  Dasha ends on  06 Oct 1980  |  Ra  Antardasha ends on  06 Oct 1980
----------- Error of  0  days in  5844.0  day Dasha
Sa  Dasha ends on  07 Oct 1999  |  Sa  Antardasha ends on  10 Oct 1983
Sa  Dasha ends on  07 Oct 1999  |  Me  Antardasha ends on  19 Jun 1986
Sa  Dasha ends on  07 Oct 1999  |  Ke  Antardasha ends on  29 Jul 1987
Sa  Dasha ends on  07 Oct 1999  |  Ve  Antardasha ends on  28 Sep 1990
Sa  Dasha ends on  07 Oct

#Gochar Chart

## Gochar Data Preparation

In [ ]:
# This cell to be executed ONLY after natal chart has been created, otherwise errors
# 
!sed -n 1p peopleData.csv > peopleData2.csv                                  # Copy the Header data
print(p21.gName, p21.SubMoonLong)

#p21.Gochar = True 
#if p21.Gochar :

now = datetime.now(pytz.timezone('Asia/Kolkata'))
timeTxt = now.strftime("%-d,%-m,%Y,%-H:%-M")
GocharTxt = 'x,'+timeTxt+',5.5,25.43,81.85,IST,standard,'+p21.gName+',nil,nil,nil,nil,nil,nil'                  # Using Lat, Long of Calcutta for Gochar
print(GocharTxt)
with open("peopleData2.csv", 'a') as file1:
    file1.write(GocharTxt)
    

_Gochar 325.406
x,9,5,2023,14:49,5.5,25.43,81.85,IST,standard,_Gochar,nil,nil,nil,nil,nil,nil


In [ ]:
# Load CSV data into Pandas dataframe
#
df = pd.read_csv('peopleData2.csv')
#df.head()

p21.AnalysisType = 'Rashi'                                     # one of ['Rashi','Navamsa']
#p21.AnalysisType = 'Navamsa'                                    # one of ['Rashi','Navamsa']
p21.ChartType = p21.AnalysisType

p21swe.C61_Cast2JSON(df)
print(p21.GLon['La'],p21.GLon['Mo'])

1 records processed, so far
1  records generated and stored in file peopleData.json
325.406 251.581


## Process Gochar Chart

In [ ]:
%%capture

chart = StoreRetrieveChart()
ReportFile = genChart('Bengal')

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)

In [ ]:
#!cat peopleData.json

# Leftovers

In [ ]:
#
#Get List of Persons whose charts are found in the database
#
selCond = {}                                              # All Charts
#pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
#pd.DataFrame(list(p21.kollection.find(selCond,{'pid.name':1,'_id':0})))
pd.DataFrame(list(p21.kollection.find(selCond,{'pid.name':1})))
#pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))


,_id,pid
0,645797a4004d072716dd4d8c,{'name': '_Gochar'}


In [ ]:
#
# Download last Report File
#
files.download(ReportFile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!rm *.doc